In [ ]:
"""
Created on Mon Apr 28 14:15 2025

Create a BedMachine-similar file for ElmerIce-geometries as input for Christoph

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf

In [ ]:
%matplotlib qt5

In [ ]:
inputpath_ElmerIce='/bettik/burgardc/DATA/SUMMER_PAPER/raw/ELMERICE_GEOMETRIES/'
outputpath_ElmerIce='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ELMERICE_NEWGEO/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
map_lim = [-3000000,3000000]

In [ ]:
file_BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_BedMachine_orig_cut = uf.cut_domain_stereo(file_BedMachine_orig, map_lim, map_lim)

In [ ]:
yyear = 2025
eexp = '104' #'104'

In [ ]:
mask_file_raw = xr.open_dataset(inputpath_ElmerIce + 'ISMIP6_GL1-SSP585_IPSL-CM6A-LR_MAR_EXP'+eexp+'_Y'+str(yyear)+'.4km.nc')

In [ ]:
mask_file_new = mask_file_raw.copy().transpose('y','x')

In [ ]:
ground_conc = (mask_file_new['gmask'] + 1)/2
isf_conc = 1 - ground_conc

In [ ]:
mask_file_new['thickness'] = mask_file_new['surface'] - mask_file_new['base']
mask_file_new['ground_conc'] = ground_conc.where(np.isfinite(ground_conc), 0)
mask_file_new['isf_conc'] = isf_conc.where(np.isfinite(isf_conc), 0)
mask_file_new['icefree_conc'] = mask_file_new['isf_conc'] * 0
mask_file_new['bed'] = mask_file_new['bedrock']

In [ ]:
isf_mask = (mask_file_new['isf_conc'] > 0).astype(int)
ground_mask_0_1 = (mask_file_new['ground_conc'] == 1).astype(int) 
ground_mask_0_2 = ground_mask_0_1.where(ground_mask_0_1 == 0, 2)
mask_0_1_2 = isf_mask + ground_mask_0_2

Correct the mask for Berkner Island

In [ ]:
xx = file_BedMachine_orig_cut.x
yy = file_BedMachine_orig_cut.y
mask_Berkner_Island = file_BedMachine_orig_cut['mask_0_1_2'].where((xx >= -1e6) & (xx <= -7.26e5) & (yy >= 5.8e5) & (yy <= 9.4e5))

In [ ]:
mask_0_1_2_corr = mask_0_1_2.where(mask_Berkner_Island != 2, 2)
mask_0_1_2_corr = mask_0_1_2_corr.where(mask_Berkner_Island != 1, 2)
mask_0_1_2_corr = mask_0_1_2_corr.where(~((mask_0_1_2_corr == 2) & (mask_0_1_2 == 1)), 1)

In [ ]:
mask_file_new['mask_0_1_2'] = mask_0_1_2_corr

In [ ]:
if eexp == '004':
    mask_file_new[['mask_0_1_2', 'ground_conc', 'isf_conc', 'icefree_conc', 'bed', 'surface', 'thickness', 'base']].to_netcdf(outputpath_ElmerIce+'ElmerIce_4km_allvars_'+str(yyear)+'.nc')
elif eexp == '104':
    mask_file_new[['mask_0_1_2', 'ground_conc', 'isf_conc', 'icefree_conc', 'bed', 'surface', 'thickness', 'base']].to_netcdf(outputpath_ElmerIce+'ElmerIce_4km_allvars_'+str(yyear)+'_constantSMB.nc')

In [ ]:
ground_conc.plot()